## 5.horseIDとjockeyIDの取得

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm
import functions
import numpy as np
import importlib

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 30

In [2]:

df = pd.read_pickle('2.2.add.日付_df.pickle')
race_id_list = df.index.unique()
print(type(race_id_list[0]))
# race_id_list = ["202101010101","202101010102"]
race_id_list
# df

<class 'str'>


Index(['202101010101', '202101010102', '202101010103', '202101010104',
       '202101010105', '202101010106', '202101010107', '202101010108',
       '202101010109', '202101010110',
       ...
       '202102010303', '202102010304', '202102010305', '202102010306',
       '202102010307', '202102010308', '202102010309', '202102010310',
       '202102010311', '202102010312'],
      dtype='object', length=108)

In [3]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      print(f"レースID: {_race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [4]:
results_new = scrape_race_results(race_id_list)
results_new

100%|██████████| 108/108 [02:33<00:00,  1.42s/it]


{'202101010101':       horse_id jockey_id
 0   2018105875     01181
 1   2018105258     01140
 ..         ...       ...
 14  2018104780     01144
 15  2018100737     05203
 
 [16 rows x 2 columns],
 '202101010102':       horse_id jockey_id
 0   2018105205     00666
 1   2018100974     01161
 ..         ...       ...
 12  2018105770     01172
 13  2018100706     01140
 
 [14 rows x 2 columns],
 '202101010103':       horse_id jockey_id
 0   2018100403     01157
 1   2018105166     01182
 ..         ...       ...
 10  2018102272     01015
 11  2018103483     01096
 
 [12 rows x 2 columns],
 '202101010104':       horse_id jockey_id
 0   2018100537     01182
 1   2018103489     01096
 ..         ...       ...
 13  2018101597     01176
 14  2018104222     01124
 
 [15 rows x 2 columns],
 '202101010105':       horse_id jockey_id
 0   2019104536     01157
 1   2019103120     01176
 ..         ...       ...
 11  2019106524     01182
 12  2019105105     00666
 
 [13 rows x 2 columns],
 '20210101

----

## 6.取得したデータを成形して元のデータと結合

In [5]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
print(results_new)
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)
# results

{'202101010101':                 horse_id jockey_id
202101010101  2018105875     01181
202101010101  2018105258     01140
...                  ...       ...
202101010101  2018104780     01144
202101010101  2018100737     05203

[16 rows x 2 columns], '202101010102':                 horse_id jockey_id
202101010102  2018105205     00666
202101010102  2018100974     01161
...                  ...       ...
202101010102  2018105770     01172
202101010102  2018100706     01140

[14 rows x 2 columns], '202101010103':                 horse_id jockey_id
202101010103  2018100403     01157
202101010103  2018105166     01182
...                  ...       ...
202101010103  2018102272     01015
202101010103  2018103483     01096

[12 rows x 2 columns], '202101010104':                 horse_id jockey_id
202101010104  2018100537     01182
202101010104  2018103489     01096
...                  ...       ...
202101010104  2018101597     01176
202101010104  2018104222     01124

[15 rows x 2 columns],

In [6]:
print(type(results))
print(len(results))
print(type(df))
print(len(df))

<class 'pandas.core.frame.DataFrame'>
1366
<class 'pandas.core.frame.DataFrame'>
1366


In [7]:
# pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [8]:
# df_add_id.to_pickle("df_add_id.pickle")

----

## 7.idのリストを作成

In [114]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
# horse_id_list = horse_id_list[:50]
# horse_id_list = ['2018105875',"2018102192"]

In [10]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [159]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付',encoding='EUC-JP')[0]
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [166]:
horse_results = scrape_horse_results(horse_id_list)

100%|██████████| 2/2 [00:04<00:00,  2.26s/it]


In [167]:
horse_results

{'2018105875':             日付    開催  天気   R          レース名  映像  頭数  枠番  馬番   オッズ  人気  着順  \
 0   2022/01/16  1小倉2   曇  10   小郡特別(2勝クラス) NaN  12   7  10   3.5   2  10   
 1   2021/12/26  6阪神8   曇  12  猪名川特別(2勝クラス) NaN  15   2   2   8.2   4   6   
 ..         ...   ...  ..  ..           ...  ..  ..  ..  ..   ...  ..  ..   
 5   2021/06/12  1札幌1  小雨   1         3歳未勝利 NaN  16   5   9  13.5   7   1   
 6   2021/04/10  1新潟1   晴   5         3歳未勝利 NaN  16   8  16  13.7   6   6   
 
       騎手  斤量     距離 馬場 馬場指数     タイム   着差 ﾀｲﾑ指数   通過        ペース    上り  \
 0   秋山稔樹  54  芝1200  良   **  1:09.8  1.6    **  5-6  32.8-35.4  36.5   
 1   亀田温心  54  芝1400  良   **  1:21.9  0.2    **  2-3  35.3-34.8  34.8   
 ..   ...  ..    ... ..  ...     ...  ...   ...  ...        ...   ...   
 5   秋山稔樹  52  芝1200  良   **  1:08.7 -0.1    **  4-4  33.8-34.9  34.6   
 6   亀田温心  53  芝1200  良   **  1:10.4  0.6    **  3-3  34.9-34.9  35.2   
 
          馬体重  厩舎ｺﾒﾝﾄ  備考   勝ち馬(2着馬)     賞金  
 0    474(+2)     NaN NaN     メメントモリ

----

## 取得したデータを成形

In [168]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [169]:
# pd.options.display.max_rows = None

results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38
2018105875,2022/01/16,1小倉2,曇,10,小郡特別(2勝クラス),NaN,12,7,10,3.5,2,10,秋山稔樹,54,芝1200,良,**,1:09.8,1.6,**,5-6,32.8-35.4,36.5,474(+2),NaN,NaN,メメントモリ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018105875,2021/12/26,6阪神8,曇,12,猪名川特別(2勝クラス),NaN,15,2,2,8.2,4,6,亀田温心,54,芝1400,良,**,1:21.9,0.2,**,2-3,35.3-34.8,34.8,472(-6),NaN,NaN,エルカスティージョ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018102192,2020/11/08,3福島4,晴,1,2歳未勝利,NaN,16,2,4,12.3,5,7,秋山稔樹,52,ダ1150,良,**,1:11.9,2.8,**,9-10,31.6-37.5,38.9,452(-2),NaN,NaN,タガノペカ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018102192,2020/10/11,4新潟2,晴,5,2歳新馬,NaN,15,7,13,25.1,9,3,秋山稔樹,52,ダ1200,良,**,1:13.9,0.4,**,1-1,36.2-37.3,37.7,454(0),NaN,NaN,リヴウェル,180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# results.to_csv("3.2.horse_vs_data.csv")

In [17]:
# time.sleep(2)
# df = pd.read_csv("3.2.horse_vs_data.csv")

In [18]:
# print(df.columns)
# df = df.iloc[:,:29]
# print(df.columns)

----

## データの自動成形

In [91]:
importlib.reload(functions)
# arr = [6,7,184,340,341,722,813,814]
arr = [
       40,492,1198,1199,1761,1762,2115,2116,2117,
       1944,1945,2305,2306,2450,2451,2485,2498,2499,
       2560,2588,2589,2590,2591,2592,2680,2681,2682,
       2683,2684,2685,2686,2687,2688,2689,3215,3216,
       3842,3843,4404,
       
       # 75,83,84,249,250,391,392,504,689,690,
       # 691,692,693,1155,1156,1363,1365,1567,
       # 1568,1600,1601,1748,1749,1825,1826,
       # 1882,1883,1899,1890
       ]
clean_index = list(map(lambda x: x-1,arr))
# l,l_length,columns_len = Molding(clean_index,"3.2.horse_vs_data.csv")

l,l_length,columns_len,おかしいリスト = functions.Molding_new(clean_index,"3.2.horse_vs_data.csv")
おかしいリスト

["4482,10,['2016100161', '2020/06/21', '3東京6', '曇', '9.0', '青梅特別(2勝クラス)', '', '16.0', '3.0', '5', '211.6', '14', '13', '石川裕紀', '54', 'ダ1600', '稍', '**', '1:38.3', '2.3', '**', '15-15', '', '16.0', '6', '11.0', '33.0', '10', '14.0', '石川裕紀', '57.0', 'ダ1800', '良', '**', '1:54.3', '1.6', '**', '5-5-5-8', '37.1-38.3', '39.5', '454(-10)', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'ティターヌ', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

In [22]:
# add = np.array(l[1:]).reshape(l_length-1,columns_len)
# df_add = pd.DataFrame(add, columns=l[0])
# print(df_add)

# df_add.to_csv('3.2.horse_vs_data.csv"',index=False)

In [23]:
# print(df["日付"].value_counts())
# print(df["開催"].value_counts())
# print(df["天気"].value_counts())
# print(df["R"].value_counts())
# print(df["レース名"].value_counts())
# print(df["映像"].value_counts())
# print(df["頭数"].value_counts())
# print(df["枠番"].value_counts())
# print(df["馬番"].value_counts())
# print(df["オッズ"].value_counts())
# print(df["人気"].value_counts())
# print(df["着順"].value_counts())
# print(df["騎手"].value_counts())
# print(df["斤量"].value_counts())